In [75]:
import torch
import torch.nn as nn
#from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.nn.functional import softmax


class Router(nn.Module):
    def __init__(self, input_size, hidden_size, router_size):
        super().__init__()
        
        # routing layer gates
        self.G1 = Parameter(torch.Tensor(input_size, router_size))
        self.G2 = Parameter(torch.Tensor(router_size, 2))
        
        self.W1 = Parameter(torch.Tensor(input_size, hidden_size))
        self.W2 = Parameter(torch.Tensor(hidden_size, 1))
        
        self.K = Parameter(torch.Tensor(input_size,1))
    
    def forward(self, x):
        g1 = torch.sigmoid(x.matmul(self.G1))
        g2 = torch.sigmoid(g1.matmul(self.G2))
        d = softmax(g2, dim=0)
        
        # TODO this is likeley not effiecient (though maybe the JIT takes care of it magically...)
        m = torch.distributions.Categorical(d)
        path = m.sample()
        
        return g1,g2,d,x
        

In [109]:
net = Router(2,4,4)
(g1, g2, d, x) = net(torch.randn(2))
print("g1: ", g1)
print("g2: ", g2)
print("d: ", d)
print("x:", x)
print()
print("G1", net.G1)
print("G2", net.G2)



g1:  tensor([0.5000, 1.0000, 0.5000, 1.0000], grad_fn=<SigmoidBackward>)
g2:  tensor([0.5000, 1.0000], grad_fn=<SigmoidBackward>)
d:  tensor([0.3775, 0.6225], grad_fn=<SoftmaxBackward>)
x: tensor([2.1996, 0.0659])

G1 Parameter containing:
tensor([[0.0000e+00, 3.6893e+19, 0.0000e+00, 3.6893e+19],
        [9.6297e-32, 4.5911e-41, 9.6517e-32, 4.5911e-41]], requires_grad=True)
G2 Parameter containing:
tensor([[ 0.0000e+00,  3.6893e+19],
        [-1.6465e-09, -4.6577e-10],
        [-1.6444e-09, -4.6577e-10],
        [-1.6486e-09,  1.8654e-40]], requires_grad=True)
